Импортируем необходимые библиотеки


*   **requests** для получения сырых данных
*   **json** для парсинга сырых данных
*   **pandas** для трансформации данных
*   **pint** для перевода единиц измерения в необходимые
*   **re** для упрощения работы с названиями колонок таблиц


In [793]:
import requests
import json
import pandas as pd
import pint
import re

Функция **get_data** извлекает сырые данные по передаваемому url и парсит данные в формате json

In [794]:
def get_data(url):
  raw_data = requests.get(url).text
  json_data = json.loads(raw_data)
  return json_data

Изначальный url выдавал null значения, поэтому даты были изменены, изучаемый интервал **16.06.2025 - 30.06.2025**

In [795]:
data = get_data("https://api.open-meteo.com/v1/forecast?latitude=55.0344&longitude=82.9434&daily=sunrise,sunset,daylight_duration&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,wind_speed_80m,wind_direction_10m,wind_direction_80m,visibility,evapotranspiration,weather_code,soil_temperature_0cm,soil_temperature_6cm,rain,showers,snowfall&timezone=auto&timeformat=unixtime&wind_speed_unit=kn&temperature_unit=fahrenheit&precipitation_unit=inch&start_date=2025-06-16&end_date=2025-06-30")

Извлекаем первый элемент из данных - **общая информация** путем копирования всех данных и удаления ненужных элементов

In [796]:
general_data = data.copy()
for el in ['hourly_units', 'hourly', 'daily_units', 'daily']:
  general_data.pop(el)
print (general_data)

{'latitude': 55.0, 'longitude': 83.0, 'generationtime_ms': 42.08958148956299, 'utc_offset_seconds': 25200, 'timezone': 'Asia/Novosibirsk', 'timezone_abbreviation': 'GMT+7', 'elevation': 135.0}


Для читаемости таблиц создаем функцию, которая **добавит единицы измерения в основные таблицы** на основе переименования колонок

In [797]:
def append_units(df, measurment_units):
  new_cols = {}
  for col in df.columns:
    unit = measurment_units.get(col)
    new_cols[col] = f"{col} ({unit})"
  return df.rename(columns=new_cols)

hourly_df = append_units(pd.DataFrame(data['hourly']), data['hourly_units'])
daily_df = append_units(pd.DataFrame(data['daily']), data['daily_units'])

print (hourly_df)
print (daily_df)

     time (unixtime)  temperature_2m (°F)  relative_humidity_2m (%)  \
0         1750006800                 69.5                        88   
1         1750010400                 67.6                        86   
2         1750014000                 66.6                        86   
3         1750017600                 65.6                        87   
4         1750021200                 64.9                        88   
..               ...                  ...                       ...   
355       1751284800                 73.4                        64   
356       1751288400                 72.7                        66   
357       1751292000                 71.4                        71   
358       1751295600                 69.7                        79   
359       1751299200                 68.1                        87   

     dew_point_2m (°F)  apparent_temperature (°F)  temperature_80m (°F)  \
0                 65.8                       73.4                  68.1 

Делаем преобразования с **unixtime величинами** в обеих таблицах, учитываем временную зону

In [798]:
def get_datetime(df):

  tz = general_data['timezone']

  if 'sunrise (unixtime)' in df.columns:
    df['sunrise (unixtime)'] = pd.to_datetime(df['sunrise (unixtime)'],
                                              unit='s',
                                              utc=True).dt.tz_convert(tz)
    df['sunset (unixtime)'] = pd.to_datetime(df['sunset (unixtime)'],
                                             unit='s',
                                             utc=True).dt.tz_convert(tz)
    df.rename(columns={
        'sunrise (unixtime)': 'sunrise (datetime)',
        'sunset (unixtime)': 'sunset (datetime)'}, inplace=True)
    df['time (unixtime)'] = pd.to_datetime(df['time (unixtime)'], unit='s',
                                           utc=True).dt.tz_convert(tz).dt.date
  else:
    df['time (unixtime)'] = pd.to_datetime(df['time (unixtime)'], unit='s',
                                           utc=True).dt.tz_convert(tz)

  return df.rename(columns={'time (unixtime)': 'datetime'})

hourly_df = get_datetime(hourly_df)
daily_df = get_datetime(daily_df)

print (hourly_df)
print (daily_df)

                     datetime  temperature_2m (°F)  relative_humidity_2m (%)  \
0   2025-06-16 00:00:00+07:00                 69.5                        88   
1   2025-06-16 01:00:00+07:00                 67.6                        86   
2   2025-06-16 02:00:00+07:00                 66.6                        86   
3   2025-06-16 03:00:00+07:00                 65.6                        87   
4   2025-06-16 04:00:00+07:00                 64.9                        88   
..                        ...                  ...                       ...   
355 2025-06-30 19:00:00+07:00                 73.4                        64   
356 2025-06-30 20:00:00+07:00                 72.7                        66   
357 2025-06-30 21:00:00+07:00                 71.4                        71   
358 2025-06-30 22:00:00+07:00                 69.7                        79   
359 2025-06-30 23:00:00+07:00                 68.1                        87   

     dew_point_2m (°F)  apparent_temper

Используя библиотеку pint создаем функции, которые будут **преобразовывать необходимые для нас величины**, применяем на таблице с почасовыми данными

In [799]:
ureg = pint.UnitRegistry()

def f_to_c(f_temp):
  temp = ureg.Quantity(f_temp, ureg.degF)
  return round(temp.to(ureg.degC).magnitude, 1)

def kn_to_mps(speed_kn):
  speed = speed_kn * ureg.kt
  return round(speed.to(ureg.m / ureg.sec).magnitude, 1)

def ft_to_m(ft_dist):
  dist = ft_dist * ureg.ft
  return round(dist.to(ureg.m).magnitude, 3)

def inch_to_mm(inch_prec):
  prec = inch_prec * ureg.inch
  return round(prec.to(ureg.mm).magnitude, 3)

def sec_to_h(sec_dur):
  dur = sec_dur * ureg.sec
  return round(dur.to(ureg.hr).magnitude, 2)

def convert_measurments(df):
  f_cols = df.columns[df.columns.str.contains('°F')]
  kn_cols = df.columns[df.columns.str.contains('kn')]
  ft_cols = df.columns[df.columns.str.contains('ft')]
  inch_cols = df.columns[df.columns.str.contains('inch')]

  for col in f_cols:
    df[col] = df[col].apply(f_to_c)

  for col in kn_cols:
    df[col] = df[col].apply(kn_to_mps)

  for col in ft_cols:
    df[col] = df[col].apply(ft_to_m)

  for col in inch_cols:
    df[col] = df[col].apply(inch_to_mm)

  df.rename(columns={col: col.replace('°F', 'celsius') for col in f_cols},
            inplace=True)
  df.rename(columns={col: col.replace('kn', 'm_per_s') for col in kn_cols},
            inplace=True)
  df.rename(columns={col: col.replace('ft', 'm') for col in ft_cols},
            inplace=True)
  df.rename(columns={col: col.replace('inch', 'mm') for col in inch_cols},
            inplace=True)

convert_measurments(hourly_df)

print (hourly_df)

                     datetime  temperature_2m (celsius)  \
0   2025-06-16 00:00:00+07:00                      20.8   
1   2025-06-16 01:00:00+07:00                      19.8   
2   2025-06-16 02:00:00+07:00                      19.2   
3   2025-06-16 03:00:00+07:00                      18.7   
4   2025-06-16 04:00:00+07:00                      18.3   
..                        ...                       ...   
355 2025-06-30 19:00:00+07:00                      23.0   
356 2025-06-30 20:00:00+07:00                      22.6   
357 2025-06-30 21:00:00+07:00                      21.9   
358 2025-06-30 22:00:00+07:00                      20.9   
359 2025-06-30 23:00:00+07:00                      20.1   

     relative_humidity_2m (%)  dew_point_2m (celsius)  \
0                          88                    18.8   
1                          86                    17.4   
2                          86                    16.8   
3                          87                    16.4   
4     

Создаем функцию, которая **приводит таблицу к финальному виду**. Для этого в качестве индекса используем datetime данные и считаем среднее по каждому дню, округляя полученные величины до 3 знаков после запятой, параллельно избавляемся от ненужных столбцов и оставляем только дату в качестве индекса

In [800]:
def finalize_table(df):

  df.set_index('datetime', inplace = True)

  df_fin = df.resample('D').mean().round(3).drop(columns=
   ['wind_direction_10m (°)',	'wind_direction_80m (°)',
    'evapotranspiration (mm)', 'weather_code (wmo code)',
    'soil_temperature_0cm (celsius)', 'soil_temperature_6cm (celsius)'])
  df_fin.index = df_fin.index.date

  return df_fin

final_df_daily_avg = finalize_table(hourly_df)

print (final_df_daily_avg)

            temperature_2m (celsius)  relative_humidity_2m (%)  \
2025-06-16                    22.167                    67.417   
2025-06-17                    22.800                    62.167   
2025-06-18                    24.825                    58.958   
2025-06-19                    26.433                    56.708   
2025-06-20                    24.012                    60.458   
2025-06-21                    20.429                    74.375   
2025-06-22                    22.354                    56.625   
2025-06-23                    22.929                    54.833   
2025-06-24                    19.029                    73.458   
2025-06-25                    17.329                    66.250   
2025-06-26                    17.975                    50.917   
2025-06-27                    20.004                    49.667   
2025-06-28                    21.004                    61.542   
2025-06-29                    20.808                    72.583   
2025-06-30

Определяем функцию для **изменения имени колонок**, на вход также передаем параметр, в данном коде используется 24h или daylight. Если в имени колонке есть выражения вида {число}m или (m), то добавляем avg в начало названия, если (datetime), то возвращаем имя колонки без изменений, ко всем другим вариантам в начале ставим префикс total. Удаляем единицы измерения в ()

In [801]:
def change_col_name(col, param):
  if re.search(r'\d+m|\((m)\)', col):
    new = re.sub(r'\s*\(.*\)', '', col)
    return f'avg_{new}_{param}'
  elif re.search(r'\((datetime)\)|\((s)\)', col):
    return col
  else:
    new = re.sub(r'\s*\(.*\)', '', col)
    return f'total_{new}_{param}'

final_df_daily_avg.rename(columns=lambda col: change_col_name(col, '24h'), inplace=True)

print (final_df_daily_avg)

            avg_temperature_2m_24h  avg_relative_humidity_2m_24h  \
2025-06-16                  22.167                        67.417   
2025-06-17                  22.800                        62.167   
2025-06-18                  24.825                        58.958   
2025-06-19                  26.433                        56.708   
2025-06-20                  24.012                        60.458   
2025-06-21                  20.429                        74.375   
2025-06-22                  22.354                        56.625   
2025-06-23                  22.929                        54.833   
2025-06-24                  19.029                        73.458   
2025-06-25                  17.329                        66.250   
2025-06-26                  17.975                        50.917   
2025-06-27                  20.004                        49.667   
2025-06-28                  21.004                        61.542   
2025-06-29                  20.808              

Переходим ко второй смысловой части - определение среднего значения параметров во время светового дня. Так как у нас есть данные за каждый час, а световой день начинается и заканчивается в определенные минуты и даже секунды, то **применим интерполяцию и предположим, какие могли быть значения в каждую минуту**, если данные изменяются линейно. Можно было бы просто округлить минуты к ближайшим часам, но захотелось применить более интересную логику, точность результатов между двумя подходами можно проверить отдельно

In [802]:
minute_df = hourly_df.resample('min').asfreq().interpolate(method='time')
minute_df['date'] = minute_df.index.date
minute_df = minute_df.reset_index()
print (minute_df)

                       datetime  temperature_2m (celsius)  \
0     2025-06-16 00:00:00+07:00                 20.800000   
1     2025-06-16 00:01:00+07:00                 20.783333   
2     2025-06-16 00:02:00+07:00                 20.766667   
3     2025-06-16 00:03:00+07:00                 20.750000   
4     2025-06-16 00:04:00+07:00                 20.733333   
...                         ...                       ...   
21536 2025-06-30 22:56:00+07:00                 20.153333   
21537 2025-06-30 22:57:00+07:00                 20.140000   
21538 2025-06-30 22:58:00+07:00                 20.126667   
21539 2025-06-30 22:59:00+07:00                 20.113333   
21540 2025-06-30 23:00:00+07:00                 20.100000   

       relative_humidity_2m (%)  dew_point_2m (celsius)  \
0                     88.000000               18.800000   
1                     87.966667               18.776667   
2                     87.933333               18.753333   
3                     87.900000

Объединяем полученную таблицу с одной из исходных, где есть данные о восходе и закате, применяем фильтрацию, чтобы **оставить только строки с необходимыми значеними**, убираем ненужные колонки и переименовываем колонку datetime

In [803]:
daylight_df = minute_df.merge(daily_df, left_on='date', right_on='datetime')

daylight_df = daylight_df[(daylight_df['datetime_x'] >=
                           daylight_df['sunrise (datetime)']) &
                           (daylight_df['datetime_x'] <=
                            daylight_df['sunset (datetime)'])].drop(
                                columns=['date', 'datetime_y'])

daylight_df.rename(columns=lambda x: x.replace('_x', ''), inplace=True)

print (daylight_df)

                       datetime  temperature_2m (celsius)  \
289   2025-06-16 04:49:00+07:00                 17.891667   
290   2025-06-16 04:50:00+07:00                 17.883333   
291   2025-06-16 04:51:00+07:00                 17.875000   
292   2025-06-16 04:52:00+07:00                 17.866667   
293   2025-06-16 04:53:00+07:00                 17.858333   
...                         ...                       ...   
21486 2025-06-30 22:06:00+07:00                 20.820000   
21487 2025-06-30 22:07:00+07:00                 20.806667   
21488 2025-06-30 22:08:00+07:00                 20.793333   
21489 2025-06-30 22:09:00+07:00                 20.780000   
21490 2025-06-30 22:10:00+07:00                 20.766667   

       relative_humidity_2m (%)  dew_point_2m (celsius)  \
289                   90.450000                   16.30   
290                   90.500000                   16.30   
291                   90.550000                   16.30   
292                   90.600000

Обращаемся к **ранее написанным функциям** для приведения таблицы к необходимому виду

In [804]:
final_df_dailight_avg = finalize_table(daylight_df)
final_df_dailight_avg.rename(columns=lambda col:
                             change_col_name(col, 'daylight'), inplace=True)
print (final_df_dailight_avg)

            avg_temperature_2m_daylight  avg_relative_humidity_2m_daylight  \
2025-06-16                       23.055                             61.778   
2025-06-17                       24.061                             56.749   
2025-06-18                       26.269                             53.651   
2025-06-19                       28.261                             49.708   
2025-06-20                       24.160                             57.909   
2025-06-21                       20.797                             71.812   
2025-06-22                       23.895                             49.343   
2025-06-23                       23.534                             50.364   
2025-06-24                       19.300                             70.806   
2025-06-25                       18.028                             61.343   
2025-06-26                       19.100                             45.487   
2025-06-27                       21.321                         

**Объединяем таблицы со средним за 24 часа и средним за период светового дня** для получения финальной таблицы с ежедневными данными

In [805]:
final_daily_df = final_df_daily_avg.merge(final_df_dailight_avg,
                                          left_index = True, right_index=True)
final_daily_df = final_daily_df.reset_index()
final_daily_df.rename(columns={'index': 'datetime'}, inplace=True)

Исправляем **порядок колонок**, который должен быть задан в финальной таблице

In [806]:
cols = final_daily_df.columns.tolist()
print(cols)

['datetime', 'avg_temperature_2m_24h', 'avg_relative_humidity_2m_24h', 'avg_dew_point_2m_24h', 'total_apparent_temperature_24h', 'avg_temperature_80m_24h', 'avg_temperature_120m_24h', 'avg_wind_speed_10m_24h', 'avg_wind_speed_80m_24h', 'avg_visibility_24h', 'total_rain_24h', 'total_showers_24h', 'total_snowfall_24h', 'avg_temperature_2m_daylight', 'avg_relative_humidity_2m_daylight', 'avg_dew_point_2m_daylight', 'total_apparent_temperature_daylight', 'avg_temperature_80m_daylight', 'avg_temperature_120m_daylight', 'avg_wind_speed_10m_daylight', 'avg_wind_speed_80m_daylight', 'avg_visibility_daylight', 'total_rain_daylight', 'total_showers_daylight', 'total_snowfall_daylight', 'sunrise (datetime)', 'sunset (datetime)', 'daylight_duration (s)']


In [807]:
cols = ['datetime', 'avg_temperature_2m_24h', 'avg_relative_humidity_2m_24h',
        'avg_dew_point_2m_24h', 'total_apparent_temperature_24h',
        'avg_temperature_80m_24h', 'avg_temperature_120m_24h',
        'avg_wind_speed_10m_24h', 'avg_wind_speed_80m_24h',
        'avg_visibility_24h', 'total_rain_24h', 'total_showers_24h',
        'total_snowfall_24h', 'avg_temperature_2m_daylight',
        'avg_relative_humidity_2m_daylight', 'avg_dew_point_2m_daylight',
        'total_apparent_temperature_daylight', 'avg_temperature_80m_daylight',
        'avg_temperature_120m_daylight', 'avg_wind_speed_10m_daylight',
        'avg_wind_speed_80m_daylight', 'avg_visibility_daylight',
        'total_rain_daylight', 'total_showers_daylight',
        'total_snowfall_daylight', 'daylight_duration (s)',
        'sunrise (datetime)', 'sunset (datetime)']
final_daily_df = final_daily_df[cols]

print (final_daily_df)

      datetime  avg_temperature_2m_24h  avg_relative_humidity_2m_24h  \
0   2025-06-16                  22.167                        67.417   
1   2025-06-17                  22.800                        62.167   
2   2025-06-18                  24.825                        58.958   
3   2025-06-19                  26.433                        56.708   
4   2025-06-20                  24.012                        60.458   
5   2025-06-21                  20.429                        74.375   
6   2025-06-22                  22.354                        56.625   
7   2025-06-23                  22.929                        54.833   
8   2025-06-24                  19.029                        73.458   
9   2025-06-25                  17.329                        66.250   
10  2025-06-26                  17.975                        50.917   
11  2025-06-27                  20.004                        49.667   
12  2025-06-28                  21.004                        61

**Форматируем**: переводим секунды в часы, время восхода и заката представляем в iso формате. Таблица с ежедневными данными готова

In [808]:
def convert_daylight_measurments(df):

  df['daylight_duration (s)'] = df['daylight_duration (s)'].apply(sec_to_h)
  dt_cols = df.columns[df.columns.str.contains(r'\(datetime\)')]

  for col in dt_cols:
    df[col] = df[col].dt.tz_convert('UTC').dt.strftime('%Y-%m-%dT%H:%M:%SZ')

  df.rename(columns={col: col.replace(r'\(datetime\)', '_iso') for col
                     in dt_cols}, inplace=True)
  df.rename(columns={'daylight_duration (s)': 'daylight_hours'}, inplace=True)

convert_daylight_measurments(final_daily_df)

print (final_daily_df)

      datetime  avg_temperature_2m_24h  avg_relative_humidity_2m_24h  \
0   2025-06-16                  22.167                        67.417   
1   2025-06-17                  22.800                        62.167   
2   2025-06-18                  24.825                        58.958   
3   2025-06-19                  26.433                        56.708   
4   2025-06-20                  24.012                        60.458   
5   2025-06-21                  20.429                        74.375   
6   2025-06-22                  22.354                        56.625   
7   2025-06-23                  22.929                        54.833   
8   2025-06-24                  19.029                        73.458   
9   2025-06-25                  17.329                        66.250   
10  2025-06-26                  17.975                        50.917   
11  2025-06-27                  20.004                        49.667   
12  2025-06-28                  21.004                        61

Представлем datetime в iso формате, определяем **нужный порядок колонок** в таблице с почасовыми данными

In [812]:
final_hourly_df = hourly_df.reset_index()

final_hourly_df['datetime_iso'] = final_hourly_df['datetime'].dt.tz_convert(
    'UTC').dt.strftime('%Y-%m-%dT%H:%M:%SZ')

cols = final_hourly_df.columns.tolist()
print(cols)

['datetime', 'temperature_2m (celsius)', 'relative_humidity_2m (%)', 'dew_point_2m (celsius)', 'apparent_temperature (celsius)', 'temperature_80m (celsius)', 'temperature_120m (celsius)', 'wind_speed_10m (m_per_s)', 'wind_speed_80m (m_per_s)', 'wind_direction_10m (°)', 'wind_direction_80m (°)', 'visibility (m)', 'evapotranspiration (mm)', 'weather_code (wmo code)', 'soil_temperature_0cm (celsius)', 'soil_temperature_6cm (celsius)', 'rain (mm)', 'showers (mm)', 'snowfall (mm)', 'datetime_iso']


In [813]:
cols = ['datetime', 'datetime_iso', 'wind_speed_10m (m_per_s)',
        'wind_speed_80m (m_per_s)','temperature_2m (celsius)',
        'apparent_temperature (celsius)', 'temperature_80m (celsius)',
        'temperature_120m (celsius)', 'soil_temperature_0cm (celsius)',
        'soil_temperature_6cm (celsius)', 'rain (mm)', 'showers (mm)',
        'snowfall (mm)']
final_hourly_df = final_hourly_df[cols]

print (final_hourly_df)

                     datetime          datetime_iso  wind_speed_10m (m_per_s)  \
0   2025-06-16 00:00:00+07:00  2025-06-15T17:00:00Z                       2.1   
1   2025-06-16 01:00:00+07:00  2025-06-15T18:00:00Z                       2.3   
2   2025-06-16 02:00:00+07:00  2025-06-15T19:00:00Z                       2.2   
3   2025-06-16 03:00:00+07:00  2025-06-15T20:00:00Z                       1.6   
4   2025-06-16 04:00:00+07:00  2025-06-15T21:00:00Z                       1.5   
..                        ...                   ...                       ...   
355 2025-06-30 19:00:00+07:00  2025-06-30T12:00:00Z                       1.0   
356 2025-06-30 20:00:00+07:00  2025-06-30T13:00:00Z                       1.3   
357 2025-06-30 21:00:00+07:00  2025-06-30T14:00:00Z                       1.3   
358 2025-06-30 22:00:00+07:00  2025-06-30T15:00:00Z                       1.0   
359 2025-06-30 23:00:00+07:00  2025-06-30T16:00:00Z                       1.0   

     wind_speed_80m (m_per_

**Извлекаем единицы измерения из скобок**

In [814]:
def rename_col(col):
  return re.sub(r'\s*\(([^)]*)\)', r'_\1', col)

final_hourly_df.columns = [rename_col(col) for col in final_hourly_df.columns]

print (final_hourly_df)

                     datetime          datetime_iso  wind_speed_10m_m_per_s  \
0   2025-06-16 00:00:00+07:00  2025-06-15T17:00:00Z                     2.1   
1   2025-06-16 01:00:00+07:00  2025-06-15T18:00:00Z                     2.3   
2   2025-06-16 02:00:00+07:00  2025-06-15T19:00:00Z                     2.2   
3   2025-06-16 03:00:00+07:00  2025-06-15T20:00:00Z                     1.6   
4   2025-06-16 04:00:00+07:00  2025-06-15T21:00:00Z                     1.5   
..                        ...                   ...                     ...   
355 2025-06-30 19:00:00+07:00  2025-06-30T12:00:00Z                     1.0   
356 2025-06-30 20:00:00+07:00  2025-06-30T13:00:00Z                     1.3   
357 2025-06-30 21:00:00+07:00  2025-06-30T14:00:00Z                     1.3   
358 2025-06-30 22:00:00+07:00  2025-06-30T15:00:00Z                     1.0   
359 2025-06-30 23:00:00+07:00  2025-06-30T16:00:00Z                     1.0   

     wind_speed_80m_m_per_s  temperature_2m_celsius

Записываем ежедневные и почасовые данные в **итоговую таблицу** в формате csv

In [815]:
final_daily_df.to_csv('final_table_Kolyshkina.csv', index=False)
final_hourly_df.to_csv('final_table_Kolyshkina.csv', mode='a', index=False)